<a href="https://colab.research.google.com/github/etengler/Techos/blob/main/INTERFAZ_OpenBuildings_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Buildings - spatial analysis examples

Esta notebook presenta una metodología personalizada para la descarga de la información publicada por [Open Buildings](https://sites.research.google/open-buildings/):

In [ ]:
!sudo apt-get install swig
!pip install s2geometry pygeos geopandas

##Imports

In [ ]:
import ipyleaflet
from ipyleaflet import basemaps, basemap_to_tiles
from ipywidgets import Dropdown, Layout, VBox

from ipyleaflet import Polygon, DrawControl
from shapely.geometry import shape
import shapely.wkt as wkt
from ipywidgets import Layout, VBox ,HBox, Button
import ipywidgets as widgets
from ipyleaflet import  LayersControl
from ipywidgets import HTML
import requests

In [ ]:
from IPython.display import display
import geopandas as gpd
from shapely import wkt
from ipyleaflet import  LayersControl
import json

In [ ]:
#Imports de OpenBuildings
import functools
import glob
import gzip
import multiprocessing
import os
import shutil
import tempfile
from typing import List, Optional, Tuple

from google.colab import files
#from IPython import display
from mpl_toolkits.axes_grid1 import make_axes_locatable
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s2geometry as s2
import shapely
import tensorflow as tf
import tqdm.notebook

##FUNCIONES

In [ ]:
# Función para actualizar el mapa base según la selección del dropdown
"""
def actualizar_mapa_base(cambio):
    # Remover todas las capas
    for layer in mapa.layers[1:]: #....................................................no se debe remover todo
        mapa.remove_layer(layer)
    # Agregar la nueva capa seleccionada
    mapa.add_layer(cambio['new'])
"""

# Función para actualizar los departamentos según la provincia seleccionada
def on_provincia_change(change):
    provincia_seleccionada = change["new"]
    if provincia_seleccionada is None:
        # Si no se selecciona ninguna provincia, deshabilitar el segundo desplegable
        desplegable_departamento.options = []
        desplegable_departamento.disabled = True
    elif provincia_seleccionada in provincias_dict:
        departamentos = provincias_dict[provincia_seleccionada]

        # Usar el nombre del departamento como opciones y el código como valor
        desplegable_departamento.options = [(d["Departamento"], d["Codigo"]) for d in departamentos]
        desplegable_departamento.value = None
        desplegable_departamento.disabled = False


# Función para manejar el evento de cambio de selección  ??
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("Seleccionaste:", change['new'])


#PASAJE A WKT
def pasar_a_wkt(geometria): #de geoJson a WKT
  geometry = shape(geometria['geometry'])

  wkt_result = geometry.wkt  # Devolverá una cadena con la representación WKT

  #print("WKT:", wkt_result)
  return wkt_result


def armarUrlWkt(provincia):
# Construimos la URL del archivo WKT en GitHub, usando la provincia seleccionada
# Usar el enlace "raw" para una descarga directa
  url_base_github = "https://raw.githubusercontent.com/etengler/Techos/499436897f89ae7fc8aa47963413ab38b31a3d51/wkt_por_provincia/"
  archivo_nombre = f"{provincia}_geometries.wkt"  # Nombre del archivo basado en la provincia
  url_wkt = f"{url_base_github}{archivo_nombre}"  # URL completa
  #print(url_wkt)
  return url_wkt


# Función para obtener la geometría del departamento basado en el código
def obtener_geometria(codigo, contenido):
    # Eliminar espacios y saltos de línea para consistencia
    codigo_str = str(codigo).strip()  # Asegúrate de que el código es una cadena y sin espacios
    for i, line in enumerate(contenido):
        if f"in1: {codigo_str}" in line:
            # Asegúrate de que la siguiente línea contiene la geometría
            if i + 1 < len(contenido):
                return contenido[i + 1].strip()  # Obtener la geometría WKT
    return None  # Si no se encuentra la geometría


def calcular(b):
  #Variables globales
  global valor_seleccionado
  global geojson_layer

  # 1 ---- SELECCION POR PROVINCIA
  if boton_radio.value == 'Provincia':
    clave_seleccionada = desplegable_provincia.value
    valor_seleccionado = diccionario.get(clave_seleccionada) #!
    """
    if valor_seleccionado is not None:
      print(f"El valor asociado a '{clave_seleccionada}' es correcto")
    else:
      print("La clave ingresada no existe en el diccionario.")
     """
  # 2 ---- SELECCION POR DEPARTAMENTO
  if boton_radio.value == 'Departamento':
    clave_seleccionada = desplegable_provincia.value #valor de provincia

    # Obtener el contenido del archivo desde la URL de la provincia seleccionada
    response = requests.get(armarUrlWkt(clave_seleccionada))

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
      wkt_content = response.text.split("\n")  # Dividir el contenido por líneas
    else:
      print("Error al obtener el archivo WKT. Código de estado:", response.status_code)

    # Obtener el código del departamento desde el desplegable
    codigo_departamento = desplegable_departamento.value  # Este es el código que necesitas usar

  # Obtener la geometría WKT del departamento seleccionado basado en el código
    valor_seleccionado = obtener_geometria(codigo_departamento, wkt_content)
    """
    if valor_seleccionado is not None:
      pass
      print("Geometría del departamento seleccionado:", valor_seleccionado)
    else:
      print("No se encontró la geometría para el código dado:", codigo_departamento)
    """

  # 3 ---- DIBUJAR A MANO
  else:
    obtener_ultima_geometria = dc.last_draw
    estado = dc.last_action

    # Verificar si se ha dibujado una geometría
    if obtener_ultima_geometria['geometry'] is not None and estado != 'deleted':
      #label_errores.value = "llaala"
      valor_seleccionado = pasar_a_wkt(obtener_ultima_geometria)
      #print('valor seleccionado: ', valor_seleccionado) #valor seleccionado vale distinto ya que paso por pasar a WKT

    else:
    # Aquí puedes manejar el caso en el que no se ha dibujado ninguna geometría
      label_errores.value = "Por favor, dibuje una geometría en el mapa."

  label_errores.value = "Por favor, espere mientras se procesa..."
  corre_openBuildings() #OpenBuildings
  convertir_geometria_shape()
  label_errores.value = "Listo!"


#----------------------------------------------------------------------------------------


##interfaz y area

In [ ]:
#VARIABLES GLOBALES
valor_seleccionado = None #geometria/area
obtener_ultima_geometria = None
geojson_layer = None
edificios = None #
edificios_70 = None

In [ ]:

# Crear un mapa centrado en una ubicación específica
mapa = ipyleaflet.Map(center=(-40, -64), zoom=4, layout=Layout(height='700px'), scroll_wheel_zoom=True)

############################################################# DESPLEGABLE
# Crear diferentes mapas base
base_osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
basemap_satelital=basemap_to_tiles(basemap=basemaps.Esri.WorldImagery)

# Inicialmente usar OpenStreetMap
#mapa.add_layer(base_osm)
mapa.add_layer(basemap_satelital)


# Crear un menú desplegable para elegir el mapa base
dropdown = Dropdown(
    options={
        "OpenStreetMap": base_osm,
        "Mapa Satelital": basemap_satelital,
    },
    value=base_osm,
    description="Elige un mapa base:",
    #layout=Layout(width='max-content')
    layout=Layout(width='45%')
)

############################################################# HERRAMIENTA DIBUJO
dc = DrawControl(
    marker={},
    rectangle={},
    circle={},
    circlemarker={},
    polygon={"shapeOptions": {"color": "#0000FF", "fillColor":"#0000FF" }},
    polyline={},
)

mapa.add(dc)

############################################################# RADIO BUTTON
# Define las opciones del botón de radio
opcionesProv = ['Provincia', 'Departamento', 'Dibujo en el mapa']

# Crea el botón de radio
boton_radio = widgets.RadioButtons(
    options=opcionesProv,
    description='Elija la forma de selección del área:',
    disabled=False
)

############################################################# HERRAMIENTA CAPAS
# Agregar el control de capas
layer_control = LayersControl(position='topright')
mapa.add_control(layer_control)

############################################################# Interfaz grafica
#------------- PASOS -----------#
# Crear el widget Label con un título
#label_titulo = widgets.Label("Título 1")

#label_titulo.style = {"font-size": "50px"}  # ANCHO

# Crear un widget HTML con formato personalizado para el tamaño del texto
label_titulo = HTML("<h1>Título 1</h1>")

#label_1 = widgets.Label("1. Dibuje un área de interes en el mapa. ")
label_errores = widgets.Label("...")

label_espacio_blanco = widgets.Label(" ")
label_linea = widgets.Label(value='___________________________________________________________________')

label_area = widgets.Label("Si elige como opción Provincia o Departamento puede filtrarlo a continuación:")



#----------  Botón  ----------#
# Establece el ancho del botón
button_width = '120px'  # Puedes ajustar el valor según tus necesidades

boton_calcular = widgets.Button(
    description='CALCULAR', ##ver
    button_style='primary',
    tooltip='Click en el botón para calcular huellas de edificios',
    layout=widgets.Layout(width=button_width)
)

boton_descargar= widgets.Button(
    description='DESCARGAR', ##ver
    button_style='primary',
    tooltip='Click en el botón para descargar huellas de edificios',
    layout=widgets.Layout(width=button_width)
)

"""
boton_cargar_resultado_mapa = widgets.Button(
    description='CARGAR RESULTADO', ##ver
    button_style='primary',
    tooltip='Click en el botón para cargar el resultado en el mapa',
    layout=widgets.Layout(width=button_width)
)
"""
#VERTICALL
#hbox_botones = HBox ([boton_calcular, boton_descargar])
# Crear un contenedor para el mapa y el dropdown
#vbox = VBox([mapa,label_1, hbox_botones])#dropdown
# Mostrar el mapa y el menú desplegable
#display(vbox)


# Mostrar el mapa y el menú desplegable
#display(hbox_2)


In [ ]:
#DESPLEGABLES

# URL del archivo JSON en GitHub
url_departamentos = "https://raw.githubusercontent.com/etengler/Techos/main/departamentos_por_provincia.json"

# Obtener el contenido del archivo desde la URL
response = requests.get(url_departamentos)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Cargar el contenido como JSON
    provincias_dict = json.loads(response.text)
    #print("Archivo JSON cargado con éxito")
else:
    print("Error al cargar el archivo JSON. Código de estado:", response.status_code)


style = {'description_width': 'initial'}


# Crear el primer desplegable con las provincias, pero con una opción vacía inicial
provincias = list(provincias_dict.keys())
opciones_provincia = [] + provincias
desplegable_provincia = widgets.Dropdown(options=opciones_provincia, description="Provincia: ", value=None, style=style)



# Crear el segundo desplegable para los departamentos, inicialmente vacío
desplegable_departamento = widgets.Dropdown(description="Departamento: ", disabled=True, style=style)

# Conectar la función al dropdown para actualizar el mapa base
#dropdown.observe(actualizar_mapa_base, 'value')

# Vincular el cambio del primer desplegable a la función de actualización
desplegable_provincia.observe(on_provincia_change, names="value")

In [ ]:
# Crear un widget vacío para espacio
espacio = widgets.Label(value='', layout=widgets.Layout(width='20px'))

#HORIZONTAL
hbox_botones = HBox ([boton_calcular,espacio, boton_descargar], layout=widgets.Layout(justify_content='center'))


# Crear un contenedor para el mapa y el dropdown
vbox = VBox([label_titulo, label_linea, boton_radio, label_espacio_blanco, label_area, desplegable_provincia, desplegable_departamento, label_espacio_blanco, hbox_botones, label_errores])#dropdown

hbox_2 = HBox ([mapa,vbox])

In [ ]:

#print(calcular())
valor_seleccionado = boton_calcular.on_click(calcular)
#print(valor_seleccionado)
#label_error_geometria_no_dibujo = widgets.Label("Por favor, dibuje una geometría en el mapa.")

##OPEN BUILDINGS


In [ ]:
region_border_source = 'Natural Earth (High Res 10m)'

region = ''

your_own_wkt_polygon = None

data_type = 'polygons'

In [ ]:
BUILDING_DOWNLOAD_PATH = ('gs://open-buildings-data/v3/'
                          f'{data_type}_s2_level_6_gzip_no_header')

def get_filename_and_region_dataframe(
    region_border_source: str, region: str,
    your_own_wkt_polygon: str) -> Tuple[str, gpd.geodataframe.GeoDataFrame]:
  """Returns output filename and a geopandas dataframe with one region row."""

  global data_type

  if your_own_wkt_polygon:
    filename = f'open_buildings_v3_{data_type}_your_own_wkt_polygon.csv.gz'
    region_df = gpd.GeoDataFrame(
        geometry=gpd.GeoSeries.from_wkt([your_own_wkt_polygon]),
        crs='EPSG:4326')
    if not isinstance(region_df.iloc[0].geometry,
                      shapely.geometry.polygon.Polygon) and not isinstance(
                          region_df.iloc[0].geometry,
                          shapely.geometry.multipolygon.MultiPolygon):
      raise ValueError("`your_own_wkt_polygon` must be a POLYGON or "
                      "MULTIPOLYGON.")
    print(f'Preparing your_own_wkt_polygon.')
    return filename, region_df

  if not region:
    raise ValueError('Please select a region or set your_own_wkt_polygon.')

  if region_border_source == 'Natural Earth (Low Res 110m)':
    url = ('https://www.naturalearthdata.com/http//www.naturalearthdata.com/'
           'download/110m/cultural/ne_110m_admin_0_countries.zip')
    !wget -N {url}
    display.clear_output()
    region_shapefile_path = os.path.basename(url)
    source_name = 'ne_110m'
  elif region_border_source == 'Natural Earth (High Res 10m)':
    url = ('https://www.naturalearthdata.com/http//www.naturalearthdata.com/'
           'download/10m/cultural/ne_10m_admin_0_countries.zip')
    !wget -N {url}
    display.clear_output()
    region_shapefile_path = os.path.basename(url)
    source_name = 'ne_10m'
  elif region_border_source == 'World Bank (High Res 10m)':
    url = ('https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/'
           '779551/wb_countries_admin0_10m.zip')
    !wget -N {url}
    !unzip -o {os.path.basename(url)}
    display.clear_output()
    region_shapefile_path = 'WB_countries_Admin0_10m'
    source_name = 'wb_10m'

  region_iso_a3 = region.split(' ')[0]
  filename = (f'open_buildings_v3_{data_type}_'
              f'{source_name}_{region_iso_a3}.csv.gz')
  region_df = gpd.read_file(region_shapefile_path).query(
      f'ISO_A3 == "{region_iso_a3}"').dissolve(by='ISO_A3')[['geometry']]
  print(f'Preparing {region} from {region_border_source}.')
  return filename, region_df


def get_bounding_box_s2_covering_tokens(
    region_geometry: shapely.geometry.base.BaseGeometry) -> List[str]:
  region_bounds = region_geometry.bounds
  s2_lat_lng_rect = s2.S2LatLngRect_FromPointPair(
      s2.S2LatLng_FromDegrees(region_bounds[1], region_bounds[0]),
      s2.S2LatLng_FromDegrees(region_bounds[3], region_bounds[2]))
  coverer = s2.S2RegionCoverer()
  # NOTE: Should be kept in-sync with s2 level in BUILDING_DOWNLOAD_PATH.
  coverer.set_fixed_level(6)
  coverer.set_max_cells(1000000)
  return [cell.ToToken() for cell in coverer.GetCovering(s2_lat_lng_rect)]


def s2_token_to_shapely_polygon(
    s2_token: str) -> shapely.geometry.polygon.Polygon:
  s2_cell = s2.S2Cell(s2.S2CellId_FromToken(s2_token, len(s2_token)))
  coords = []
  for i in range(4):
    s2_lat_lng = s2.S2LatLng(s2_cell.GetVertex(i))
    coords.append((s2_lat_lng.lng().degrees(), s2_lat_lng.lat().degrees()))
  return shapely.geometry.Polygon(coords)


def download_s2_token(
    s2_token: str, region_df: gpd.geodataframe.GeoDataFrame) -> Optional[str]:
  """Downloads the matching CSV file with polygons for the `s2_token`.

  NOTE: Only polygons inside the region are kept.
  NOTE: Passing output via a temporary file to reduce memory usage.

  Args:
    s2_token: S2 token for which to download the CSV file with building
      polygons. The S2 token should be at the same level as the files in
      BUILDING_DOWNLOAD_PATH.
    region_df: A geopandas dataframe with only one row that contains the region
      for which to keep polygons.

  Returns:
    Either filepath which contains a gzipped CSV without header for the
    `s2_token` subfiltered to only contain building polygons inside the region
    or None which means that there were no polygons inside the region for this
    `s2_token`.
  """

  global region

  s2_cell_geometry = s2_token_to_shapely_polygon(s2_token)
  region_geometry = region_df.iloc[0].geometry
  prepared_region_geometry = shapely.prepared.prep(region_geometry)
  # If the s2 cell doesn't intersect the country geometry at all then we can
  # know that all rows would be dropped so instead we can just return early.
  if not prepared_region_geometry.intersects(s2_cell_geometry):
    return None
  try:
    # Using tf.io.gfile.GFile gives better performance than passing the GCS path
    # directly to pd.read_csv.
    with tf.io.gfile.GFile(
        os.path.join(BUILDING_DOWNLOAD_PATH, f'{s2_token}_buildings.csv.gz'),
        'rb') as gf:
      # If the s2 cell is fully covered by country geometry then can skip
      # filtering as we need all rows.
      if prepared_region_geometry.covers(s2_cell_geometry):
        with tempfile.NamedTemporaryFile(mode='w+b', delete=False) as tmp_f:
          shutil.copyfileobj(gf, tmp_f)
          return tmp_f.name
      # Else take the slow path.
      # NOTE: We read in chunks to save memory.
      csv_chunks = pd.read_csv(
          gf, chunksize=2000000, dtype=object, compression='gzip', header=None)
      tmp_f = tempfile.NamedTemporaryFile(mode='w+b', delete=False)
      tmp_f.close()
      for csv_chunk in csv_chunks:
        points = gpd.GeoDataFrame(
            geometry=gpd.points_from_xy(csv_chunk[1], csv_chunk[0]),
            crs='EPSG:4326')
        # sjoin 'within' was faster than using shapely's 'within' directly.
        points = gpd.sjoin(points, region_df, predicate='within')
        csv_chunk = csv_chunk.iloc[points.index]
        csv_chunk.to_csv(
            tmp_f.name,
            mode='ab',
            index=False,
            header=False,
            compression={
                'method': 'gzip',
                'compresslevel': 1
            })
      return tmp_f.name
  except tf.errors.NotFoundError:
    return None


def corre_openBuildings():# correr todas las funciones de openBuildings
  global edificios #aca vale none ?
  global region_border_source
  global region
  global valor_seleccionado
  global data_type
  global your_own_wkt_polygon

  your_own_wkt_polygon = valor_seleccionado
  print('your_own_wkt_polygon: ', your_own_wkt_polygon)

  # Clear output after pip install.
  #display.clear_output()
  filename, region_df = get_filename_and_region_dataframe(region_border_source,
                                                        region,
                                                        your_own_wkt_polygon)
# Remove any old outputs to not run out of disk.
  for f in glob.glob('/tmp/open_buildings_*'):
    os.remove(f)
  # Write header to the compressed CSV file.
  with gzip.open(f'/tmp/{filename}', 'wt') as merged:
    if data_type == "polygons":
      merged.write(','.join([
          'latitude', 'longitude', 'area_in_meters', 'confidence', 'geometry',
          'full_plus_code'
      ]) + '\n')
    else:
      merged.write(','.join([
        'latitude', 'longitude', 'area_in_meters', 'confidence',
        'full_plus_code'
    ]) + '\n')
  download_s2_token_fn = functools.partial(download_s2_token, region_df=region_df)
  s2_tokens = get_bounding_box_s2_covering_tokens(region_df.iloc[0].geometry)
  # Downloads CSV files for relevant S2 tokens and after filtering appends them
  # to the compressed output CSV file. Relies on the fact that concatenating
  # gzipped files produces a valid gzip file.
  # NOTE: Uses a pool to speed up output preparation.
  with open(f'/tmp/{filename}', 'ab') as merged:
    with multiprocessing.Pool(4) as e:
      for fname in tqdm.notebook.tqdm(
          e.imap_unordered(download_s2_token_fn, s2_tokens),
          total=len(s2_tokens)):
        if fname:
          with open(fname, 'rb') as tmp_f:
            shutil.copyfileobj(tmp_f, merged)
          os.unlink(fname)

  edificios = pd.read_csv(
    f"/tmp/{filename}", engine="c",
    usecols=['latitude', 'longitude', 'area_in_meters', 'confidence','geometry'])

  print(f"Se crearon {len(edificios):,} registros.")

##exportar a google drive

In [ ]:
def convertir_geometria_shape():
  global geojson_layer #cambia aca valor
  global edificios_70 #vale none
  global edificios #deberia valer

  print('edificios: ', edificios)
  # Convierte la columna wkt a geometría
  edificios['geom'] = edificios['geometry'].apply(wkt.loads)

  # Creamos un GeoDataFrame
  edificios_gdf = gpd.GeoDataFrame(edificios, geometry='geom')

  # Define el código EPSG de la proyección original (4326 en este caso)
  original_crs = 'EPSG:4326'

  # Asigna el CRS al GeoDataFrame
  edificios_gdf.crs = original_crs

  # Define el código EPSG al que deseas reproyectar (3857 en este caso)
  target_crs = 'EPSG:3857'

  # Reproyecta el GeoDataFrame
  edificios_gdf = edificios_gdf.to_crs(target_crs)

  edificios_gdf.drop(columns='geometry')



  edificios_70 = edificios_gdf.query(f"confidence > 0.70")
  #print(edificios_70)



  # Convertir el GeoDataFrame a GeoJSON para ipyleaflet
  geojson_data = edificios_70.__geo_interface__  # Convierte el GeoDataFrame a GeoJSON

  # Crear una capa GeoJSON
  geojson_layer = ipyleaflet.GeoJSON(data=geojson_data, name="Edificios") #cambia valor

In [ ]:
def carga_resultado_mapa():
  global geojson_layer

  mapa.add_layer(geojson_layer)

##DESCARGA

In [ ]:
# Inicializamos Google Drive
#from google.colab import drive
#drive.mount('drive')

In [ ]:
def descarga(b):
  global geojson_layer
  global edificios_70

  from google.colab import drive
  drive.mount('drive')

  # Verificar si la variable edificios_70 es distinta de None
  if edificios_70 is not None:


    # Guarda el GeoDataFrame en un archivo shapefile si no es None
    edificios_70.to_file('drive/My Drive/test.shp', driver='ESRI Shapefile')

    print('Comienza la descarga')
  else:
      print('Error: edificios_70 es None. No se puede descargar.')

In [ ]:
boton_descargar.on_click(descarga)

#.on_click(carga_resultado_mapa)

#INTERFAZ

In [ ]:
display(hbox_2)